# Xavier initialization的理解 #

### 1、方差公式 ###

假设X、Y是独立的随机变量，则：

$E(XY) = EXEY $

有,
$D(XY) = (EX)^2DY + (EY)^2DX + DXDY  $

推导如下：
$$
\begin{aligned}
D(XY) &= E(XY)^2 - (E(XY))^2  \\ 
&= E(X^2Y^2) - (EXEY)^2 \\
&= EX^2EY^2 - (EX)^2(EY)^2 \\
\end{aligned}
$$
同时，
$$
\begin{aligned}
(EX)^2DY + (EY)^2DX + DXDY &= (EX)^2[EY^2 - (EY)^2] + (EY)^2[EX^2 - (EX)^2] + [EX^2 - (EX)^2] * [E(Y^2) - (EY)^2] \\
&= EX^2EY^2 - (EX)^2(EY)^2 \\
&= D(XY)
\end{aligned}
$$
因此，当$EX = EY = 0$时，$D(XY) = DXDY$

### 2、问题 ###

假设一个神经元，输入为n维的向量X，权重为W，输出为Y：
$$Y = W_1X_1 + W_2X_2 + ... + W_nX_n$$

$W_i$与$X_i$相互独立，当$E(W_i) = 0, E(X_i) = 0$时，
$$D(W_iX_i) = D(W_i)D(X_i)$$

同时假设$W_i$和$X_i$都是独立同分布的，则

$$D(Y) = nD(W_i)D(X_i)$$

即： $$\frac{D(Y)}{D(X_i)} = nD(W_i)$$

我们希望输入和输出的方差保持不变，那么：

$$nD(W_i) = 1$$
即：
$$D(W_i) = \frac{1}{n}$$

其中$n$是输入神经元个数，
对于均匀分布$ U[a,b]$，其方差是：
$$\frac{(b-a)^2}{12}$$

传统的初始化方法是用均匀分布$U[-\frac{1}{\sqrt{n}},\frac{1}{\sqrt{n}}]$初始化$W_i$
这样$W_i$的方差是$\frac{1}{3n}$

此时：
$$nD(W_i) = \frac{1}{3}$$
使用$U[-\frac{\sqrt{3}}{\sqrt{n}},\frac{\sqrt{3}}{\sqrt{n}}]$初始化$W_i$能使得$D(W_i) = \frac{1}{n}$

Bengio对此进行了改进，假设输入输入神经元个数为$n_{in}$，输出神经元个数为$n_{out}$，
**normalized initialization**为：
$$W \sim U\left[-\frac{\sqrt{6}}{\sqrt{n_{in}+n_{out}}},\frac{\sqrt{6}}{\sqrt{n_{in}+n_{out}}}\right]$$

此时，
$$D(W_i) = \frac{2}{n_{in}+n_{out}}$$
此公式，考虑了反向传播时输入输出的方差，即要求：
$$D(W_i) = \frac{1}{n_{out}}$$
折中之后即得到上面的公式


### 多层网络方差
假设神经网络的第$j$层输出为$Z^{(j)}$
则：
$$
\begin{aligned}
D\left[Z^{(j)}\right] &= n^{(j-1)}D[W^{(j-1)}]D[Z^{(j-1)}] \\
&= n^{(j-1)}D[W^{(j-1)}] * n^{(j-2)}D[W^{(j-2)}]D[Z^{(j-2)}] \\
&= ... \\
&= D[X^{(0)}]\prod_{j'=0}^{j-1}n^{(j')}D[W^{(j')}]
\end{aligned}
$$
其中$D[X^{(0)}]$及输入数据的方差$DX$

反向传播的方差同理：
假设神经网络最后一共有$d$层，从最后一层忘前传播梯度时有：

$$
D\left[\frac{\partial{Loss}}{\partial{s^{(j)}}}\right] = D\left[\frac{\partial{Loss}}{\partial{s^{(d)}}}\right] \prod_{j'=j}^{d}n^{(j'+1)}D[W^{(j')}]
$$
其中，
$$s^{(j)} = Z^{(j)}W^{(j)} + b^{(j)}$$
,
$$
\begin{aligned}
\frac{\partial{Loss}}{\partial{W^{(j)}}} &= \frac{\partial{Loss}}{\partial{s^{(j)}}} * \frac{\partial{s^{(j)}}}{\partial{W^{(j)}}} \\
&= Z^{(j)}\frac{\partial{Loss}}{\partial{s^{(j)}}}
\end{aligned}
$$
因此，
$$
\begin{aligned}
D\left[\frac{\partial{Loss}}{\partial{W^{(j)}}}\right] &= D\left[Z^{(j)}\right]D\left[\frac{\partial{Loss}}{\partial{s^{(j)}}}\right] \\
&= D[X^{(0)}]*D\left[\frac{\partial{Loss}}{\partial{s^{(d)}}}\right] * \prod_{j'=0}^{j-1}n^{(j')}D[W^{(j')}] \prod_{j'=j}^{d}n^{(j'+1)}D[W^{(j')}]
\end{aligned}
$$

为了保证前向和后向传播每层方差一致，需满足：
$$n^{(j)}D[W^{(j)}] = 1$$
$$n^{(j+1)}D[W^{(j)}] = 1$$
权衡后的结果是：

$$D\left[W^{(j)}\right] = \frac{2}{n^{(j)} + n^{(j+1)}}$$

### Reference ###

Understanding the difficulty of training deep feedforward neural networks